In [25]:
import pandas as pd
import re
import datetime
import numpy as np

In [9]:
DB_path= "Data/Excels/Database.xlsx"
def load_DB(DB_path):
    Sheets= pd.read_excel(DB_path, sheet_name=["FMSI", "Kits", "Shim_crossing", "Kit_crossing", "SHIMS"])
    FMSI= Sheets['FMSI']
    SHIMS= Sheets['SHIMS']
    Kits= Sheets['Kits']
    Shim_crossing= Sheets['Shim_crossing']
    Kit_crossing= Sheets['Kit_crossing']

    FMSI= FMSI.astype(str)
    Shim_crossing= Shim_crossing.astype(str)

    Flat_SHIMS= FMSI.merge(Shim_crossing, on= 'FMSI').merge(SHIMS, on= 'SHIM PN')
    Flat_kits= FMSI.merge(Kit_crossing, on= 'FMSI').merge(Kits, on= 'KIT PN')
    return Flat_SHIMS, Flat_kits, FMSI, SHIMS, Kits, Shim_crossing, Kit_crossing

Flat_SHIMS, Flat_kits, FMSI, SHIMS, Kits, Shim_crossing, Kit_crossing= load_DB(DB_path)

In [10]:
results=dict()
for line in FMSI['Application(s)']:
    print('\n\n\n\n\nLine: ', line)
    groups= line.split(' | ')
    for group in groups:
        if group.startswith('Front: ') or group.startswith('Rear: '):
            location= group.split(':')[0]
            group= group[7:]
            brands= group.split('; ')
            for brand in brands:
                print('\tBrand: ', brand)
                cars= brand.split(', ')
                for i ,car in enumerate(cars):
                    if i==0:
                        constructor= car.split()[0]
                        car= car[len(constructor):]
                    years= re.findall("(\d+-\d+)", car)
                    if not years:
                        years= re.findall("(\d{4})", car)
                    if not years:
                        years= re.findall("(\d{2})", car)
                    if not years:
                        years= [""]
                    years= years[0]
                    car= car.replace(years, "")
                    print('\t\tLocation :', location)
                    print('\t\tConstructor: ', constructor)
                    print('\t\tCar: ', car)
                    print("\t\tyear: ", years, '\n')
                






Line:  Front: Cadillac Eldorado 68-67; Ford Country Squire 67-66,  Custom 67-66,  Galaxie 67-66,  LTD 1967,  Thunderbird 67-65; Lincoln Continental 69-65; Mercury Commuter 67-66,  Marquis 1967,  Montclair 67-66,  Monterey 67-66,  Park Lane 67-66
	Brand:  Cadillac Eldorado 68-67
		Location : Front
		Constructor:  Cadillac
		Car:   Eldorado 
		year:  68-67 

	Brand:  Ford Country Squire 67-66,  Custom 67-66,  Galaxie 67-66,  LTD 1967,  Thunderbird 67-65
		Location : Front
		Constructor:  Ford
		Car:   Country Squire 
		year:  67-66 

		Location : Front
		Constructor:  Ford
		Car:   Custom 
		year:  67-66 

		Location : Front
		Constructor:  Ford
		Car:   Galaxie 
		year:  67-66 

		Location : Front
		Constructor:  Ford
		Car:   LTD 
		year:  1967 

		Location : Front
		Constructor:  Ford
		Car:   Thunderbird 
		year:  67-65 

	Brand:  Lincoln Continental 69-65
		Location : Front
		Constructor:  Lincoln
		Car:   Continental 
		year:  69-65 

	Brand:  Mercury Commuter 67-66,  Marquis 

In [11]:
result= pd.DataFrame()
for idx in FMSI.index:
    FMSI_num= FMSI.loc[idx, "FMSI"]
    line= FMSI.loc[idx, "Application(s)"]

    groups= line.split(' | ')
         
    for group in groups:
        if group.startswith('Front: ') or group.startswith('Rear: '):
            location= group.split(':')[0]
            group= group[len(location)+1:]
            brands= group.split('; ')
            for brand in brands:
                cars= brand.split(', ')
                for i ,car in enumerate(cars):
                    if i==0:
                        constructor= car.split()[0]
                        car= ' '.join(car.split()[1:])
                    years= re.findall("\s(\d+-\d+)", car)
                    if not years:
                        years= re.findall("\s(\d{4})", car)
                    if not years:
                        years= re.findall("\s(\d{2})", car)
                    if not years:
                        years= [""]
                    years= years[-1]
                    car= car.replace(years, "")
                    car= car.strip()
                    r= {
                        'Line no': idx,
                        "FMSI": FMSI_num,
                        'Application(s)': line,
                        'Location': location,
                        'Constructor': constructor,
                        'Car': car,
                        'Years': years
                        }

                    result= pd.concat([result, pd.DataFrame([r])], ignore_index=True)
result.to_excel("Application_parse_v0.xlsx")

In [20]:
result[['start_date', 'end_date']] = result['Years'].str.extract(r'(\d{2,4})?-?(\d{2,4})')
result[['start_date', 'end_date']]= result[['start_date', 'end_date']].fillna('1900')

In [21]:
result[['start_date', 'end_date']].isna().sum()

start_date    0
end_date      0
dtype: int64

In [22]:
today = datetime.datetime.now()
#accessing the year attribute
this_year = today.year
this_year_2_digits= int(str(this_year)[2:])


def to_4_digit_year(year_str):
    year= int(year_str)
    if len(year_str) == 2:
        if year <= this_year_2_digits:
            return int('20' + year_str)
        else:
            return int('19' + year_str)
    else:
        return int(year_str)

In [23]:
result['start_date'] = result['start_date'].apply(to_4_digit_year)
result['end_date'] = result['end_date'].apply(to_4_digit_year)

In [30]:
result.replace({1900: np.nan}, inplace= True)

In [29]:
result.to_excel("Application_parse_v1.xlsx")